In [1]:
import pandas as pd 
import numpy as np
from torchflare.experiments import Experiment
import torchflare.callbacks as cbs 
import torchflare.metrics as metrics
from sklearn.model_selection import train_test_split
import os

import torch
import torch.nn as nn 
import torch.nn.functional as F

In [2]:
train_df = pd.read_csv("dataset/train.csv")

In [3]:
classes = train_df.label.nunique()

In [4]:
train_labels = train_df['label'].values
train_images = (train_df.iloc[:,1:].values).astype('float32')
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels,
                                                                     stratify=train_labels, random_state=123,
                                                                     test_size=0.20)



In [5]:
train_images = train_images.reshape(train_images.shape[0],1, 28, 28)
val_images = val_images.reshape(val_images.shape[0], 1,28, 28)

In [6]:
train_images = train_images / 255.0
val_images = val_images/255.0

In [7]:
class Net(torch.nn.Module):
    def __init__(self, out_features):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.ReLU(),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
        )
       
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.ReLU(),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
        )
        
        self.fc1 = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(7 * 7 * 64, 256),
            nn.BatchNorm1d(num_features=256, eps=1e-05, momentum=0.1),
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        
        self.fc2 = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(256, out_features),
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        # Flatten the 3 last dimensions (channels, width, height) to one
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [8]:
metric_list = [metrics.Accuracy(num_classes=classes, multilabel=False)]

callbacks = [
    cbs.EarlyStopping(monitor="accuracy", mode = "max",patience=5),
    cbs.ModelCheckpoint(monitor="accuracy", mode = "max"),
    cbs.ReduceLROnPlateau(mode="max" , patience = 2),
    cbs.DiscordNotifierCallback(exp_name = "MNIST-EXP" , webhook_url = os.environ.get("DISCORD_WEBHOOK") ,
                               send_figures = True)
    
]

In [11]:
exp = Experiment(
    num_epochs=3,
    fp16=True,
    device="cuda",
    seed=42,
)
exp.compile_experiment(
    module=Net, #The uninstantiated model_class for the neural network.
    module_params = {"out_features" : 10},
    optimizer="Adam",
    optimizer_params = {"lr" : 3e-4},
    callbacks=callbacks,
    criterion="cross_entropy",
    metrics=metric_list,
    main_metric="accuracy",
)

In [12]:
exp.fit(x = train_images , y = train_labels , val_data = (val_images , val_labels), batch_size = 32)


Epoch: 1/3
Train: 1050/1050 [=========================]- 165s 157ms/step - train_loss: 0.2563 - train_accuracy: 0.9377
Valid: 263/263 [=========================]- 1s 5ms/step - val_loss: 0.0517 - val_accuracy: 0.9472

Epoch: 2/3
Train: 1050/1050 [=========================]- 22s 21ms/step - train_loss: 0.0839 - train_accuracy: 0.9603
Valid: 263/263 [=========================]- 1s 6ms/step - val_loss: 0.0375 - val_accuracy: 0.9631

Epoch: 3/3
Train: 1050/1050 [=========================]- 23s 22ms/step - train_loss: 0.0639 - train_accuracy: 0.9683
Valid: 263/263 [=========================]- 2s 6ms/step - val_loss: 0.0348 - val_accuracy: 0.9696


In [13]:
exp.get_logs()

,Epoch,train_loss,train_accuracy,val_loss,val_accuracy
0,1,0.256268,0.937738,0.051720,0.947190
1,2,0.083880,0.960304,0.037531,0.963119
2,3,0.063928,0.968257,0.034790,0.969619
